In [14]:
import argparse
parser = argparse.ArgumentParser(prog = 'Run Model')
parser.add_argument('--WalkerID', type=int, choices=range(10000), default=0)
parser.add_argument('--uv_filepath', type=str, default='uv_final.npy')
parser.add_argument('--saving_location', type=str, default='')
parser.add_argument('--averages_fstring', type=str, default='averages_{}_float32.npy')
parser.add_argument('--BoxesPath', type=str, default="/amphora/bradley.greig/21CMMC_wTs_LC_RSDs_Nicolas/Programs/LightConeBoxes")
parser.add_argument('--ParametersPath', type=str, default="/amphora/bradley.greig/21CMMC_wTs_LC_RSDs_Nicolas/Programs/GridPositions")
parser.add_argument('--depth_mhz', type=int, default = 0) # if depth==0 calculate it from cube, else fix it to given value
# parser.add_argument('--uv_treshold', type=int, default = 1) # taking in account only baselines which were visited uv_treshold amount of times 
parser.add_argument('--SKA_observation_time', type=int, default = 1000)
inputs = parser.parse_args("--WalkerID 9999 --uv_filepath ../data/uv_Steven.npy --saving_location ../DatabaseTest --BoxesPath ../DatabaseTest --averages_fstring ../DatabaseTest/averages_{}_float32.npy".split(" "))

import numpy as np
import sys
import os
from src.py21cnn.database import DatabaseUtils
from src.py21cnn.formatting import Filters
import json

import matplotlib.pyplot as plt
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
%matplotlib inline

#define path to database, send to program as parameters if different from default
Redshifts = ['006.00060', '006.75589', '007.63960', '008.68274', '009.92624', '011.42503', \
            '013.25424', '015.51874', '018.36856', '022.02434', '026.82138', '033.28927', '034.50984']
Parameters = ["ZETA", "TVIR_MIN", "L_X", "NU_X_THRESH"]
database = DatabaseUtils.Database(Parameters, Redshifts, inputs.BoxesPath, inputs.ParametersPath)
deltaTmin = -250
deltaTmax = 50
Zmax = 30

print("loading lightcone")
Box = database.CombineBoxes(inputs.WalkerID)
print("removing large Z")
Box = Filters.RemoveLargeZ(Box, database, Z=Zmax)
print("removing NaNs")
np.nan_to_num(Box, copy=False, nan=deltaTmin, posinf=deltaTmax, neginf=deltaTmin)
print("clipping large values")
np.clip(Box, deltaTmin, deltaTmax, out=Box)

BoxAverage = np.load(inputs.averages_fstring.format(inputs.WalkerID))
print("removing large Z for average")
BoxAverage = Filters.RemoveLargeZ(BoxAverage, database, Z=Zmax)

Box -= BoxAverage

loading lightcone
removing large Z
removing NaNs
clipping large values
removing large Z for average


In [15]:
import tools21cm as t2c
N_ant = 512 #or 513?

t2c.const.set_hubble_h(0.678)
t2c.const.set_omega_matter(0.308)
t2c.const.set_omega_baryon(0.048425)
t2c.const.set_omega_lambda(0.692)
t2c.const.set_ns(0.968)
t2c.const.set_sigma_8(0.815)

d0 = t2c.cosmology.z_to_cdist(float(Redshifts[0]))
cdist = np.array(range(Box.shape[-1] + 1))*1.5 + d0
redshifts = t2c.cosmology.cdist_to_z(cdist)
redshifts_mean = (redshifts[:-1] + redshifts[1:]) / 2

In [16]:
def noise(Box, depth_mhz, uv, seed_index):
    finalBox = []
    for i in range(Box.shape[-1]):
        if depth_mhz == 0:
            depth_mhz = t2c.cosmology.z_to_nu(redshifts[i]) - t2c.cosmology.z_to_nu(redshifts[i+1])
        noise = t2c.noise_model.noise_map(ncells=200,
                                          z=redshifts_mean[i],
                                          depth_mhz=depth_mhz,
                                          obs_time=inputs.SKA_observation_time,
                                          boxsize=300, 
                                          uv_map=uv[..., i],
                                          N_ant=N_ant,
                                          seed = 1000000*i + 100*inputs.WalkerID + seed_index, #last index is noise number index
                                          ) # I've corrected the function so it returns noise in uv, not in real space
        noise = t2c.telescope_functions.jansky_2_kelvin(noise, redshifts_mean[i])
        finalBox.append(noise)
    finalBox = np.moveaxis(np.array(finalBox), 0, -1)
    finalBox[uv_bool] = 0
    return finalBox
def noise_n_signal(Box, depth_mhz, uv, seed_index):
    Noise = noise(Box, depth_mhz, uv, seed_index)
    return np.real(np.fft.ifft2(Box + Noise, axes=(0, 1)))

In [17]:
uv = np.load(inputs.uv_filepath)
uv_bool = (uv < 1)

Box = np.fft.fft2(Box, axes=(0, 1))
Box[uv_bool] = 0
Box = noise_n_signal(Box, 0, uv, seed_index = 0)

In [18]:
from scipy.integrate import quadrature

In [19]:
def E(z):
    return np.sqrt(t2c.const.Omega0*(1.+z)**3+t2c.const.lam)
def kappa(z, k_cube, b = 0):
    return np.sqrt(k_cube[0]**2 + k_cube[1]**2) * E(z) / (1+z) * quadrature(1/E, 0, z)[0] + b
def W(k_cube, z, b = 0):
    k_scaled = k_cube[2] / kappa(z, k_cube, b)
    return 1 - (k_scaled >= -1.) * (k_scaled <= 1.)

In [20]:
k_perp = np.fft.fftfreq(Box.shape[0], d=1.5)
k_parallel = np.fft.fftfreq(Box.shape[-1], d=1.5)
k_cube = np.meshgrid(k_perp, k_perp, k_parallel)

In [ ]:
Box_final = np.zeros(Box.shape)
Box_inv = np.fft.fftn(Box)
for z in redshifts_mean:
    